## Localizando pontos na região de saturação para serem ajustados

In [ ]:
"""
CARREANDO BIBLIOTECAS NECESSÁRIAS
"""

import numpy as np #bibloteca para manipulação de arrays e aplicações matemáticas #type: ignore
import matplotlib.pyplot as plt #biblioteca para lot de figuras #type: ignore
import os #biblioteca para manipulação de diretórios/arquivos
from caminhosDosDiretorios import diretorioDeDadosDesteDispositivo as caminhoDiretorio #variavel que guarda o caminho do diretório dos arquivos de medida

###################################
## CHAMANDO E RECONHECENDO DADOS ##
###################################


# Função para extrair o ângulo do nome do arquivo
def extrair_angulo(caminhoDiretorio):
    partes = nome_arquivo.split('_')
    angulo_str = partes[-1].split('.')[0]
    return int(angulo_str)

# Extrai o ângulo do nome do arquivo
angulo = extrair_angulo(os.path.basename(caminho_arquivo))

# Carrega os dados do arquivo, ignorando 12 linhas
dados = np.loadtxt(caminho_arquivo, skiprows=12)

# Extrai os dados de campo e magnetização
campo = dados[:, 0]
magnetizacao = dados[:, 1]

"""
DIVIDINDO A CURVA DE HISTERESE EM DUAS (SUBIDA E DESCIDA)
"""

# Separando a curva de histerese em subida e descida
subida_indices = []
descida_indices = []

# Identifica os pontos de inversão para separar subida e descida
for i in range(1, len(campo)):
    if campo[i] > campo[i - 1]:
        subida_indices.append(i)
    elif campo[i] < campo[i - 1]:
        descida_indices.append(i)

campo_subida = campo[subida_indices]
magnetizacao_subida = magnetizacao[subida_indices]

campo_descida = campo[descida_indices]
magnetizacao_descida = magnetizacao[descida_indices]

#########################################
## DEFININDO CARACTERÍSTICAS DA FIGURA ##
#########################################

# Define o tamanho da figura
fig_size = (10, 8)

# Define o tamanho da fonte dos rótulos dos eixos x e y
fonte_eixos = 14

# Define o tamanho da fonte dos títulos
fonte_titulo = 16

# Define o tamanho dos números nos eixos x e y
tamanho_numeros = 12

# Define o tamanho da fonte da legenda
tamanho_legenda = 12

# Cria uma nova figura com tamanho definido
plt.figure(figsize=fig_size)

# Plota todos os pontos da curva de subida
plt.plot(campo_subida, magnetizacao_subida, label='Subida', color='blue', marker='o', linestyle='none')

# Plota a linha da curva de subida
plt.plot(campo_subida, magnetizacao_subida, color='blue')

# Plota todos os pontos da curva de descida
plt.plot(campo_descida, magnetizacao_descida, label='Descida', color='red', marker='o', linestyle='none')

# Plota a linha da curva de descida
plt.plot(campo_descida, magnetizacao_descida, color='red')

# Adiciona linhas de grade
plt.grid(True, which='both', linestyle='-', linewidth=0.5, color='gray')
plt.axhline(0, color='black', linewidth=1)  # Linha de grade zero para magnetização
plt.axvline(0, color='black', linewidth=1)  # Linha de grade zero para campo

# Adiciona título e rótulos com tamanho de fonte personalizado
plt.xlabel('Campo (G)', fontsize=fonte_eixos)
plt.ylabel('Momentum (emu)', fontsize=fonte_eixos)
plt.title(f'Curva de histerese (Ta_CoFeB_Ag_x15) para o ângulo {angulo}°', fontsize=fonte_titulo)

# Define o tamanho dos números nos eixos x e y
plt.tick_params(axis='both', which='major', labelsize=tamanho_numeros)

# Adiciona legenda com tamanho de fonte personalizado
plt.legend(fontsize=tamanho_legenda)

"""
DESTACANDO NA FIGURA PONTOS INDENTIFICADOS ANTERIORMENTE
"""

# Identificação dos X primeiros e X últimos pontos da curva de subida
plt.scatter(campo_subida[:4], magnetizacao_subida[:4], color='blue', s=100, edgecolors='black', zorder=5)
plt.scatter(campo_subida[-6:], magnetizacao_subida[-6:], color='blue', s=100, edgecolors='black', zorder=5)

# Identificação dos X primeiros e X últimos pontos da curva de descida
plt.scatter(campo_descida[:7], magnetizacao_descida[:7], color='red', s=100, edgecolors='black', zorder=5)
plt.scatter(campo_descida[-5:], magnetizacao_descida[-5:], color='red', s=100, edgecolors='black', zorder=5)

# Exibe o gráfico
plt.show()


## Calculando médias de acordo com a necessidade observada anteriormente

In [ ]:
# Calcula as médias dos X primeiros e X últimos pontos das curvas de subida e descida
media_ultimos_subida = np.mean(magnetizacao_subida[-4:])
media_primeiros_subida = np.mean(magnetizacao_subida[:6])
media_primeiros_descida = np.mean(magnetizacao_descida[:7])
media_ultimos_descida = np.mean(magnetizacao_descida[-5:])

# Calcula as médias extras solicitadas
media_superiores = np.mean([media_ultimos_subida, media_primeiros_descida])
media_inferiores = np.mean([media_primeiros_subida, media_ultimos_descida])

# Imprime os valores das médias na tela
print("Média dos X primeiros pontos da curva de subida:", media_primeiros_subida)
print("Média dos X últimos pontos da curva de subida:", media_ultimos_subida)
print("Média dos X primeiros pontos da curva de descida:", media_primeiros_descida)
print("Média dos X últimos pontos da curva de descida:", media_ultimos_descida)
print("Média entre os X último pontos da curva de subida e os X primeiro da curva de descida:", media_superiores)
print("Média entre os X primeiro pontos da curva de subida e os X último da curva de descida:", media_inferiores)

# Armazena os valores das médias em variáveis
medias = {
    'media_primeiros_subida': media_primeiros_subida,
    'media_ultimos_subida': media_ultimos_subida,
    'media_primeiros_descida': media_primeiros_descida,
    'media_ultimos_descida': media_ultimos_descida,
    
    'media_superiores': media_superiores,
    'media_inferiores': media_inferiores
}

# Imprime as variáveis
print(medias)

## Ajuste da região de saturação de acordo com a necessidade observada

In [ ]:
# Cria uma cópia dos dados originais
campo_substituido = np.copy(campo)
magnetizacao_substituida = np.copy(magnetizacao)

# Substitui os pontos da curva superior pela média
magnetizacao_substituida[subida_indices[-5:]] = media_superiores
magnetizacao_substituida[descida_indices[:7]] = media_superiores

# Substitui os pontos da curva inferior pela média
magnetizacao_substituida[subida_indices[:4]] = media_inferiores
magnetizacao_substituida[descida_indices[-6:]] = media_inferiores

# Define o tamanho da figura
fig_size = (10, 8)

# Define o tamanho da fonte dos rótulos dos eixos x e y
fonte_eixos = 14

# Define o tamanho da fonte dos títulos
fonte_titulo = 16

# Define o tamanho dos números nos eixos x e y
tamanho_numeros = 12

# Define o tamanho da fonte da legenda
tamanho_legenda = 12

# Cria uma nova figura com tamanho definido
plt.figure(figsize=fig_size)

# Plota todos os pontos da curva de histerese com as substituições
plt.plot(campo, magnetizacao_substituida, label='Substituído', color='green', marker='o', linestyle='none')

#Plota linha ligando os pontos
plt.plot(campo, magnetizacao_substituida)

# Adiciona linhas de grade
plt.grid(True, which='both', linestyle='-', linewidth=0.5, color='gray')
plt.axhline(0, color='black', linewidth=1)  # Linha de grade zero para magnetização
plt.axvline(0, color='black', linewidth=1)  # Linha de grade zero para campo

# Adiciona título e rótulos com tamanho de fonte personalizado
plt.xlabel('Campo (G)', fontsize=fonte_eixos)
plt.ylabel('Momentum (emu)', fontsize=fonte_eixos)
plt.title(f'Curva de histerese (Ta_CoFeB_Ag_x15) para o ângulo {angulo}° com região de saturação ajustada', fontsize=fonte_titulo)

# Define o tamanho dos números nos eixos x e y
plt.tick_params(axis='both', which='major', labelsize=tamanho_numeros)

# Adiciona legenda com tamanho de fonte personalizado
#plt.legend(fontsize=tamanho_legenda)

# Exibe o gráfico
plt.show()


## SALVANDO UM NOVO .TXT COM OS DADOS AJUSTADOS

Nesse caso, o arquivo com os dados originais é preservado e um novo arquivo com os dados ajustados é criado.

Recomendo ao pesquisador criar um diretório dentro do diretório com os dados brutos e adicionar o caminho onde deseja salvar os novos arquivos .txt para esse diretório.

In [ ]:
# Caminho onde deseja salvar o novo arquivo .txt
caminho_salvar = 'C:/Winpython/WPy64-39100/notebooks/IC/CLEIZA/Ta_CoFeB_Ag_x15/Ajustado/'

# Obtém o nome do arquivo original sem o diretório
nome_arquivo_original = os.path.basename(caminho_arquivo)

# Define o novo nome do arquivo inicial com "Ajustado" no início
novo_nome_arquivo = f"Ajustado_{nome_arquivo_original}"

# Salva os dados substituídos como um novo arquivo .txt com o nome "Ajustado" no início
novo_caminho_arquivo = os.path.join(caminho_salvar, novo_nome_arquivo)
np.savetxt(novo_caminho_arquivo, np.column_stack((campo, magnetizacao_substituida)), delimiter='\t', header='Campo (G)\tMomentum (emu)', comments='')

print(f'Os dados substituídos foram salvos no arquivo: {novo_caminho_arquivo}')